Got it 👍 — instead of code fixes, you’d like a **mathematical breakdown** of what your MATLAB script is calculating.

I’ll strip away the code mechanics and write the important steps as **equations** with symbols, so you can see the mathematics behind the implementation.

---

# 📐 Major Calculations in Mathematical Form

---

### 1. **Airfoil geometry import & normalization**

The airfoil is defined by discrete coordinates:

$$
\{(x_i, y_i)\}_{i=1}^{N}
$$

where $x_i, y_i$ are normalized by chord length $c = 1$.

The set is split into **upper surface** ($x_{\text{up}}, y_{\text{up}}$) and **lower surface** ($x_{\text{lo}}, y_{\text{lo}}$) around the leading edge.

---

### 2. **Twist rotation of airfoil**

Each section is rotated by a local **twist angle** $\theta$ (in radians).
For a point $(x, y)$:

$$
\begin{bmatrix}
x' \\ y'
\end{bmatrix}
=
\begin{bmatrix}
\cos\theta & -\sin\theta \\
\sin\theta & \cos\theta
\end{bmatrix}
\begin{bmatrix}
x \\ y
\end{bmatrix}
$$

So:

$$
x' = x\cos\theta - y\sin\theta, 
\quad 
y' = x\sin\theta + y\cos\theta
$$

This is applied to both upper and lower coordinates.

---

### 3. **Chord scaling**

If the chord length at the section is $c$, then the normalized airfoil coordinates are scaled:

$$
x_{\text{scaled}} = c \cdot x', \quad y_{\text{scaled}} = c \cdot y'
$$

---

### 4. **Placement along radial stations**

Each section is placed at a radius $r$ along the blade axis.
So the final 3D coordinates of each airfoil point are:

$$
(X, Y, Z) = (r, \; x_{\text{scaled}}, \; y_{\text{scaled}})
$$

This sweeps the airfoil into a 3D blade cross-section.

---

### 5. **Leading edge circle**

At the nose of the airfoil, a **leading edge circle** of radius $R_{\text{LE}} = \frac{c}{2}$ is constructed.
For chordwise coordinate $x$:

$$
y_{\text{circle}}(x) = \sqrt{\left(\frac{c}{2}\right)^2 - x^2}
$$

This models the rounded nose shape.

---

### 6. **Twist distribution**

A twist distribution $\theta(r)$ is applied along the blade span, typically defined by interpolation between design values:

$$
\theta(r) = \theta_{\text{root}} + \left( \frac{r - r_{\text{root}}}{r_{\text{tip}} - r_{\text{root}}} \right) \cdot (\theta_{\text{tip}} - \theta_{\text{root}})
$$

---

### 7. **Chord distribution**

Similarly, chord varies along the blade radius:

$$
c(r) = c_{\text{root}} + \left( \frac{r - r_{\text{root}}}{r_{\text{tip}} - r_{\text{root}}} \right) \cdot (c_{\text{tip}} - c_{\text{root}})
$$

---

### 8. **Lift coefficient interpolation**

The code also reads **aerodynamic coefficients** (from XFOIL or other source), i.e.:

$$
C_L(\alpha, Re, M), \quad C_D(\alpha, Re, M)
$$

Then interpolates them at the chosen design angle of attack $\alpha_{\text{design}}$.

---

### 9. **Blade surface generation**

Finally, all transformed airfoil sections at different $r$ are connected into a **blade surface mesh**.

The surface is essentially:

$$
\mathcal{S}(r, \xi) = 
\begin{bmatrix}
r \\
c(r)\, (x(\xi)\cos\theta(r) - y(\xi)\sin\theta(r)) \\
c(r)\, (x(\xi)\sin\theta(r) + y(\xi)\cos\theta(r))
\end{bmatrix}
$$

where $\xi$ parameterizes the airfoil curve.



Perfect — let’s zoom in on the **performance estimation part** of your script.
While the first half is geometry, the *second half* is about turning geometry + airfoil data into **aerodynamic performance**.

---

# 🌬️ Performance Calculation Breakdown

### 1. **Airfoil aerodynamic coefficients**

From XFOIL or similar tables, the script loads:

$$
C_L(\alpha, Re, M), \quad C_D(\alpha, Re, M)
$$

where

* $C_L$ = lift coefficient,
* $C_D$ = drag coefficient,
* $\alpha$ = angle of attack,
* $Re$ = Reynolds number,
* $M$ = Mach number.

At the **design angle of attack** $\alpha_{\text{design}}$, the script interpolates values:

$$
C_{L,\text{design}}, \quad C_{D,\text{design}}
$$

---

### 2. **Local blade element forces**

At each blade radius $r$, the **sectional lift and drag forces** per unit span are:

$$
L'(r) = \tfrac{1}{2}\rho V_{\text{rel}}^2 \, c(r) \, C_L(\alpha(r)),
$$

$$
D'(r) = \tfrac{1}{2}\rho V_{\text{rel}}^2 \, c(r) \, C_D(\alpha(r)),
$$

where $V_{\text{rel}}$ is the relative wind speed seen by the blade section:

$$
V_{\text{rel}}(r) = \sqrt{ \big(V_\infty(1-a)\big)^2 + \big(\Omega r (1+a')\big)^2 }
$$

* $V_\infty$ = free-stream wind speed
* $a$ = axial induction factor (slows down axial wind)
* $a'$ = tangential induction factor (accounts for swirl)
* $\Omega$ = rotor angular speed

---

### 3. **Elemental thrust and torque**

Lift and drag are projected into **thrust** (axial) and **torque** (tangential) contributions:

$$
dT(r) = L'(r)\cos\phi - D'(r)\sin\phi
$$

$$
dQ(r) = \big(L'(r)\sin\phi + D'(r)\cos\phi \big) r
$$

where $\phi$ = inflow angle:

$$
\tan\phi = \frac{V_\infty(1-a)}{\Omega r (1+a')}
$$

---

### 4. **Integration along blade**

The contributions are integrated from root $r_{\text{hub}}$ to tip $r_{\text{tip}}$:

$$
T = B \int_{r_{\text{hub}}}^{r_{\text{tip}}} dT(r)\, dr
$$

$$
Q = B \int_{r_{\text{hub}}}^{r_{\text{tip}}} dQ(r)\, dr
$$

where $B$ = **number of blades**.

👉 **This is the first place where “three-blade” is explicit.**
For your turbine: $B = 3$.

---

### 5. **Rotor power and efficiency**

Rotor torque and angular speed give power:

$$
P = Q \, \Omega
$$

Coefficient of power (efficiency relative to wind power available):

$$
C_P = \frac{P}{\tfrac{1}{2}\rho A V_\infty^3}
$$

where $A = \pi R^2$ is swept area.

Similarly, thrust coefficient:

$$
C_T = \frac{T}{\tfrac{1}{2}\rho A V_\infty^2}
$$

---

### 6. **Design / final speed**

The script likely determines the operating tip-speed ratio:

$$
\lambda = \frac{\Omega R}{V_\infty}
$$

At the **optimal tip-speed ratio** ($\lambda_{\text{opt}}$), power coefficient $C_P$ is maximized.
For a **3-blade rotor**, typical $\lambda_{\text{opt}} \approx 6{-}8$.
Thus the final rotor speed is estimated as:

$$
\Omega_{\text{opt}} = \frac{\lambda_{\text{opt}} V_\infty}{R}
$$

---

# 🔑 Where the “three-blade” assumption enters

1. **Integration multiplier:** $B = 3$ multiplies thrust and torque integrals.
2. **Tip-speed ratio choice:** Optimal $\lambda$ depends on blade count — fewer blades → lower $\lambda_{\text{opt}}$; more blades → higher $\lambda_{\text{opt}}$. For 3 blades, $\lambda_{\text{opt}} \sim 6{-}8$.
3. **Final efficiency:** The 3-blade design balances structural loads, noise, and aerodynamic efficiency. This directly affects the predicted $C_P$ curve and the **maximum power output.**

---

✅ So in summary:
The script combines **airfoil data** + **blade geometry** + **Blade Element Momentum (BEM) equations** to compute **thrust, torque, power, and efficiency**.
The **3-blade choice** explicitly enters through $B = 3$ in force/torque sums and indirectly through the assumed **optimal tip-speed ratio** used to estimate final speed.


Yes — the same code can be modified for a **single, fixed wing blade design** (like an airplane wing or a test-section foil). You just need to strip out the parts that are *specific to a rotating wind turbine*. Let me break it down:

---

# ✅ What carries over unchanged

* **Airfoil coordinate import** → $(x_i, y_i)$ geometry.
* **Scaling by chord** $c$.
* **Applying twist** (rotation matrix).
* **Plotting the wing section(s)**.
* **Interpolating aerodynamic coefficients** from XFOIL tables.

All of this works for any lifting surface, rotating or not.

---

# ❌ What you remove or simplify

The following steps are **rotor-specific** and can be removed if we just want a fixed wing:

1. **Number of blades $B$** → irrelevant, since you only have one wing.
2. **Azimuthal/radial integration** → not needed; you won’t be sweeping an arc.
3. **Tip-speed ratio $\lambda$** and angular velocity $\Omega$ → not applicable.
4. **Induction factors $(a, a')$** from BEM → not relevant.

---

# 🧮 What remains for a fixed wing

For a fixed wing at free-stream velocity $V_\infty$:

* **Sectional lift and drag per unit span**:

$$
L'(r) = \tfrac{1}{2}\rho V_\infty^2 \, c(r) \, C_L(\alpha(r)),
$$

$$
D'(r) = \tfrac{1}{2}\rho V_\infty^2 \, c(r) \, C_D(\alpha(r)),
$$

* **Total lift and drag** (integrating along span):

$$
L = \int_{0}^{b} L'(y)\, dy, \quad
D = \int_{0}^{b} D'(y)\, dy
$$

where $b$ is the wing span.
If you keep twist and taper distributions $c(y), \alpha(y)$, then this integral still works.

* **Performance coefficients** relative to wing reference area $S = bc_\text{ref}$:

$$
C_L = \frac{L}{\tfrac{1}{2}\rho V_\infty^2 S}, \quad
C_D = \frac{D}{\tfrac{1}{2}\rho V_\infty^2 S}
$$

---

# 📊 Plots you can still make

* **Airfoil section plots** (geometry).
* **3D wing surface** built from multiple sections along the span.
* **Lift/drag coefficient distribution** along span.
* **Overall $C_L, C_D$ of the wing**.

---

# ⚠️ What you lose

Since the blade is no longer spinning:

* You **cannot compute power coefficient $C_P$**.
* No **optimum tip-speed ratio**.
* No prediction of **rotor torque or RPM**.

But you *can* still get the **aerodynamic performance of the wing** (lift, drag, efficiency $L/D$).

The modified matlab code into Python for a single, fixed wing is as follows:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# =====================================================
# USER INPUTS
# =====================================================

airfoil_file = "fx77w121.dat"    # file with airfoil coordinates (x,y) normalized by chord
cl_data_file = "fx77w121_cl.txt" # file with alpha vs CL (from XFOIL or experiment)
cd_data_file = "fx77w121_cd.txt" # file with alpha vs CD

span = 1.0       # wing span (m)
n_sections = 20  # number of spanwise stations
rho = 1.225      # air density (kg/m^3)
V_inf = 10.0     # freestream speed (m/s)
alpha_deg = 5.0  # design angle of attack (deg)

# chord distribution (example: linear taper)
c_root = 0.3
c_tip = 0.15

# twist distribution (example: linear twist from root to tip)
theta_root = 2.0   # deg
theta_tip = -2.0   # deg

# =====================================================
# GEOMETRY IMPORT
# =====================================================

# Load airfoil coordinates (x,y)
coords = np.loadtxt(airfoil_file)
x_airfoil, y_airfoil = coords[:,0], coords[:,1]

# Load polar data
alpha_cl, cl_vals = np.loadtxt(cl_data_file, unpack=True)
alpha_cd, cd_vals = np.loadtxt(cd_data_file, unpack=True)

# Interpolation functions
cl_interp = interp1d(alpha_cl, cl_vals, kind='linear')
cd_interp = interp1d(alpha_cd, cd_vals, kind='linear')

# Evaluate at design alpha
CL_section = float(cl_interp(alpha_deg))
CD_section = float(cd_interp(alpha_deg))

print(f"Section CL @ {alpha_deg}° = {CL_section:.3f}")
print(f"Section CD @ {alpha_deg}° = {CD_section:.3f}")

# =====================================================
# BUILD 3D WING
# =====================================================

y_span = np.linspace(0, span/2, n_sections)  # half-span for symmetry
c_span = np.linspace(c_root, c_tip, n_sections)
theta_span = np.linspace(theta_root, theta_tip, n_sections) * np.pi/180

wing_coords = []

for i in range(n_sections):
    c = c_span[i]
    theta = theta_span[i]
    
    # Scale and rotate airfoil
    x_scaled = c * x_airfoil
    y_scaled = c * y_airfoil
    
    x_rot = x_scaled * np.cos(theta) - y_scaled * np.sin(theta)
    z_rot = x_scaled * np.sin(theta) + y_scaled * np.cos(theta)
    
    # Place at spanwise location
    y_loc = np.full_like(x_rot, y_span[i])
    
    wing_coords.append((x_rot, y_loc, z_rot))

# =====================================================
# AERODYNAMICS: LIFT & DRAG
# =====================================================

# Sectional lift and drag per unit span
L_prime = 0.5 * rho * V_inf**2 * c_span * CL_section
D_prime = 0.5 * rho * V_inf**2 * c_span * CD_section

# Integrate across half-span, double for symmetry
L_total = 2 * np.trapz(L_prime, y_span)
D_total = 2 * np.trapz(D_prime, y_span)

S_ref = span * (c_root + c_tip) / 2  # reference area (trapezoid approx)

CL_wing = L_total / (0.5 * rho * V_inf**2 * S_ref)
CD_wing = D_total / (0.5 * rho * V_inf**2 * S_ref)

print(f"Total Lift = {L_total:.2f} N")
print(f"Total Drag = {D_total:.2f} N")
print(f"Wing CL = {CL_wing:.3f}, CD = {CD_wing:.3f}, L/D = {CL_wing/CD_wing:.1f}")

# =====================================================
# PLOTTING
# =====================================================

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')

for (x, y, z) in wing_coords:
    ax.plot(y, x, z, 'k-')

ax.set_xlabel("Span (y)")
ax.set_ylabel("Chordwise (x)")
ax.set_zlabel("Thickness (z)")
ax.set_title("3D Wing Surface")
plt.show()

# Plot lift/drag distributions
plt.figure()
plt.plot(y_span, L_prime, label="Lift per unit span")
plt.plot(y_span, D_prime, label="Drag per unit span")
plt.xlabel("Spanwise location (m)")
plt.ylabel("Force per unit span (N/m)")
plt.legend()
plt.title("Lift & Drag Distribution")
plt.show()


The following code is the same, but implements the saving portion where each slice is saved as a .sldcrv file:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import os

# =====================================================
# USER INPUTS
# =====================================================

airfoil_file = "fx77w121.dat"    # file with airfoil coordinates (x,y), normalized by chord
cl_data_file = "fx77w121_cl.txt" # file with alpha vs CL
cd_data_file = "fx77w121_cd.txt" # file with alpha vs CD

span = 1.0       # wing span (m)
n_sections = 10  # number of spanwise stations
rho = 1.225      # air density (kg/m^3)
V_inf = 10.0     # freestream speed (m/s)
alpha_deg = 5.0  # design angle of attack (deg)

# chord distribution (example: linear taper)
c_root = 0.3
c_tip = 0.15

# twist distribution (example: linear twist from root to tip)
theta_root = 2.0   # deg
theta_tip = -2.0   # deg

# output directory for SolidWorks curves
output_dir = "wing_slices"
os.makedirs(output_dir, exist_ok=True)

# =====================================================
# GEOMETRY IMPORT
# =====================================================

coords = np.loadtxt(airfoil_file)
x_airfoil, y_airfoil = coords[:,0], coords[:,1]

alpha_cl, cl_vals = np.loadtxt(cl_data_file, unpack=True)
alpha_cd, cd_vals = np.loadtxt(cd_data_file, unpack=True)

cl_interp = interp1d(alpha_cl, cl_vals, kind='linear')
cd_interp = interp1d(alpha_cd, cd_vals, kind='linear')

CL_section = float(cl_interp(alpha_deg))
CD_section = float(cd_interp(alpha_deg))

print(f"Section CL @ {alpha_deg}° = {CL_section:.3f}")
print(f"Section CD @ {alpha_deg}° = {CD_section:.3f}")

# =====================================================
# BUILD 3D WING
# =====================================================

y_span = np.linspace(0, span/2, n_sections)  # half-span for symmetry
c_span = np.linspace(c_root, c_tip, n_sections)
theta_span = np.linspace(theta_root, theta_tip, n_sections) * np.pi/180

wing_coords = []

for i in range(n_sections):
    c = c_span[i]
    theta = theta_span[i]
    
    # Scale and rotate airfoil
    x_scaled = c * x_airfoil
    y_scaled = c * y_airfoil
    
    x_rot = x_scaled * np.cos(theta) - y_scaled * np.sin(theta)
    z_rot = x_scaled * np.sin(theta) + y_scaled * np.cos(theta)
    
    # Place at spanwise location
    y_loc = np.full_like(x_rot, y_span[i])
    
    wing_coords.append((x_rot, y_loc, z_rot))
    
    # =====================================================
    # SAVE SLICE AS .SLDCRV
    # =====================================================
    filename = os.path.join(output_dir, f"slice_{i+1:02d}.sldcrv")
    with open(filename, "w") as f:
        f.write("SOLIDWORKS Curve File\n")
        for X, Y, Z in zip(x_rot, y_loc, z_rot):
            f.write(f"{X:.6f} {Y:.6f} {Z:.6f}\n")

print(f"Saved {n_sections} slices in '{output_dir}' as .SLDCRV files")

# =====================================================
# AERODYNAMICS: LIFT & DRAG
# =====================================================

L_prime = 0.5 * rho * V_inf**2 * c_span * CL_section
D_prime = 0.5 * rho * V_inf**2 * c_span * CD_section

L_total = 2 * np.trapz(L_prime, y_span)
D_total = 2 * np.trapz(D_prime, y_span)

S_ref = span * (c_root + c_tip) / 2  # reference area (trapezoid approx)

CL_wing = L_total / (0.5 * rho * V_inf**2 * S_ref)
CD_wing = D_total / (0.5 * rho * V_inf**2 * S_ref)

print(f"Total Lift = {L_total:.2f} N")
print(f"Total Drag = {D_total:.2f} N")
print(f"Wing CL = {CL_wing:.3f}, CD = {CD_wing:.3f}, L/D = {CL_wing/CD_wing:.1f}")

# =====================================================
# PLOTTING 3D WING
# =====================================================

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')

for (x, y, z) in wing_coords:
    ax.plot(y, x, z, 'k-')

ax.set_xlabel("Span (y)")
ax.set_ylabel("Chordwise (x)")
ax.set_zlabel("Thickness (z)")
ax.set_title("3D Wing Surface")
plt.show()
